In [205]:
import os, re, mmap

def generate(seedtext="the",
             weight1=1,
             weight2=2,
             weight3=3,
             weight4=4,
             unigramfile="unigramfile.dat",
             bigramfile="bigramfile.dat",
             trigramfile="trigramfile.dat",
             quadgramfile="quadgramfile.dat",
            ):
    try:
        with open(unigramfile, 'rb', 0) as file, \
            mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
            unigramtotal= text.read(text.find(b'[')).decode('utf-8')
            unigrams= text.read(text.find(b']')).decode('utf-8')
    except:
        print(f"Error reading '{unigramfile}'.")
        
    try:
        with open(bigramfile, 'rb', 0) as file, \
            mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
            bigramtotal= text.read(text.find(b'[')).decode('utf-8')
            bigrams= text.read(text.find(b']')).decode('utf-8')
    except:
        print(f"Error reading '{bigramfile}'.")
    
    try:
        with open(trigramfile, 'rb', 0) as file, \
            mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
            trigramtotal= text.read(text.find(b'[')).decode('utf-8')
            trigrams= text.read(text.find(b']')).decode('utf-8')
    except:
        print(f"Error reading '{trigramfile}'.")
        
    try:
        with open(quadgramfile, 'rb', 0) as file, \
            mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as text:
            quadgramtotal= text.read(text.find(b'[')).decode('utf-8')
            quadgrams= text.read(text.find(b']')).decode('utf-8')
    except:
        print(f"Error reading '{quadgramfile}'.")    
    
    for char in range(0,len(seedtext)):
        maybe = seedtext[:len(seedtext)-char]
        print(f"Searching unigrams for '{maybe}'.")        
        if unigrams.find(maybe) > -1:
            if unigrams[unigrams.find(maybe)-1] == "'":
                print("Found the word (or a word that starts with it).")
                currentword = unigrams[unigrams.find(maybe):unigrams.find("'",unigrams.find(maybe))]
                break
        print("Not found. Dropping a letter")
        currentword = "diffenent"           
    output=currentword
    
    print(f"Searching bigrams for '{currentword}'.")        
    if bigrams.find("'" + currentword + ' ') > -1:
        nextword = bigrams[bigrams.find("'" + currentword + ' ')+len(currentword)+2:bigrams.find("'",bigrams.find(currentword + ' '))]
        print("Found the most frequently used next word.")
    else:
        nextword = "treatment"
        print(f"Not found. Plugging in '{nextword}'.")    
    pastword =currentword
    currentword=nextword
    currentbigram=pastword + ' ' + currentword
    output+=' ' + currentword
    
    print(f"Searching trigrams for '{currentbigram}'.")  
    if trigrams.find("'" + currentbigram + ' ') > -1:
        nextword = trigrams[trigrams.find("'" + currentbigram + ' ')+len(currentbigram)+2:trigrams.find("'",trigrams.find(currentbigram + ' '))]
        print("Found the most frequently used next word.")
    else:
        nextword = "whatever"
        print(f"Not found. Plugging in '{nextword}'.")    
    pastword =currentword
    currentword=nextword
    currenttrigram=currentbigram + ' ' + currentword
    currentbigram=pastword + ' ' + currentword
    output+=' ' + currentword 

    print(f"Searching quadigrams for '{currenttrigram}'.")  
    if quadgrams.find("'" + currenttrigram + ' ') > -1:
        nextword = quadgrams[quadgrams.find("'" + currenttrigram + ' ')+len(currenttrigram)+2:quadgrams.find("'",quadgrams.find(currenttrigram + ' '))]
        print("Found the most frequently used next word.")
    else:
        nextword = "however"
        print(f"Not found. Plugging in '{nextword}'.")    
    pastword =currentword
    currentword=nextword
    currentquadgram=currenttrigram + ' ' + currentword
    currenttrigram=currentbigram + ' ' + currentword
    currentbigram=pastword + ' ' + currentword
    output+=' ' + currentword 

    
    print(f"Given '{seedtext}', our initial model generates the following: '{output}'")
      
generate()

Searching unigrams for 'the'.
Found the word (or a word that starts with it).
Searching bigrams for 'the'.
Found the most frequently used next word.
Searching trigrams for 'the same'.
Found the most frequently used next word.
Searching quadigrams for 'the same time'.
Found the most frequently used next word.
Given 'the', our initial model generates the following: 'the same time the'


In [206]:
generate("for")

Searching unigrams for 'for'.
Found the word (or a word that starts with it).
Searching bigrams for 'for'.
Found the most frequently used next word.
Searching trigrams for 'for the'.
Found the most frequently used next word.
Searching quadigrams for 'for the first'.
Found the most frequently used next word.
Given 'for', our initial model generates the following: 'for the first time'


In [207]:
generate("co")

Searching unigrams for 'co'.
Found the word (or a word that starts with it).
Searching bigrams for 'could'.
Found the most frequently used next word.
Searching trigrams for 'could not'.
Found the most frequently used next word.
Searching quadigrams for 'could not be'.
Found the most frequently used next word.
Given 'co', our initial model generates the following: 'could not be done'


In [208]:
generate("app")

Searching unigrams for 'app'.
Not found. Dropping a letter
Searching unigrams for 'ap'.
Not found. Dropping a letter
Searching unigrams for 'a'.
Found the word (or a word that starts with it).
Searching bigrams for 'and'.
Found the most frequently used next word.
Searching trigrams for 'and the'.
Found the most frequently used next word.
Searching quadigrams for 'and the same'.
Not found. Plugging in 'however'.
Given 'app', our initial model generates the following: 'and the same however'
